In [9]:
import datetime
import time
import requests
import re
from bs4 import BeautifulSoup
import csv
import os
import string
import io
import numpy as np
import pandas as pd
import matplotlib

In [10]:
import requests
from io import StringIO
import pandas as pd
import numpy as np

def crawl_price(date):
    r = requests.post('http://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + str(date).split(' ')[0].replace('-','') + '&type=ALL')
    ret = pd.read_csv(StringIO("\n".join([i.translate({ord(c): None for c in ' '}) 
                                        for i in r.text.split('\n') 
                                        if len(i.split('",')) == 17 and i[0] != '='])), header=0)
    ret = ret.set_index('證券代號')
    ret['成交金額'] = ret['成交金額'].str.replace(',','')
    ret['成交股數'] = ret['成交股數'].str.replace(',','')
    return ret

In [11]:
import datetime
import time

data = {}
n_days = 1
date = datetime.datetime.now()
fail_count = 0
allow_continuous_fail_count = 5
while len(data) < n_days:

    print('parsing', date)
    # 使用 crawPrice 爬資料
    try:
        # 抓資料
        data[date.date()] = crawl_price(date)
        print('success!')
        fail_count = 0
    except:
        # 假日爬不到
        print('fail! check the date is holiday')
        fail_count += 1
        if fail_count == allow_continuous_fail_count:
            raise
            break
    
    # 減一天
    date -= datetime.timedelta(days=1)
    time.sleep(10)

parsing 2020-09-09 10:08:32.870387
fail! check the date is holiday
parsing 2020-09-08 10:08:32.870387
success!


In [5]:
da

NameError: name 'da' is not defined

In [6]:
k[k.index=='2324']

NameError: name 'k' is not defined

In [7]:
close = pd.DataFrame({k:d['收盤價'] for k,d in data.items()}).transpose()
close.index = pd.to_datetime(close.index)
open = pd.DataFrame({k:d['開盤價'] for k,d in data.items()}).transpose()
open.index = pd.to_datetime(open.index)

high = pd.DataFrame({k:d['最高價'] for k,d in data.items()}).transpose()
high.index = pd.to_datetime(high.index)

low = pd.DataFrame({k:d['最低價'] for k,d in data.items()}).transpose()
low.index = pd.to_datetime(low.index)

volume = pd.DataFrame({k:d['成交股數'] for k,d in data.items()}).transpose()
volume.index = pd.to_datetime(volume.index)

In [8]:
stock_num='2330'
tsmc = {
    'close':close[stock_num]['2019'].dropna().astype(float),
    'open':open[stock_num]['2019'].dropna().astype(float),
    'high':high[stock_num]['2019'].dropna().astype(float),
    'low':low[stock_num]['2019'].dropna().astype(float),
    'volume': volume[stock_num]['2019'].dropna().astype(float),
}

tsmc['close'].plot()

KeyError: '2019'

In [2]:
from talib import abstract

def talib2df(talib_output):
    if type(talib_output) == list:
        ret = pd.DataFrame(talib_output).transpose()
        print('11111')
    else:
        ret = pd.Series(talib_output)
        print('22222')
    ret.index = tsmc['close'].index
    return ret;

talib2df(abstract.STOCH(tsmc)).plot()
tsmc['close'].plot(secondary_y=True)

NameError: name 'tsmc' is not defined

In [162]:
%matplotlib inline
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#專門做『技術分析』的套件
from talib import abstract
#專門抓台股的套件
import twstock

In [3]:
#設定爬蟲股票代號
sid = '0050'
#設定爬蟲時間
start = datetime.datetime.now() - datetime.timedelta(days=180)
end = datetime.date.today()
#導入pandas_datareader
from pandas_datareader import data
# 與yahoo請求，套件路徑因版本不同
pd.core.common.is_list_like = pd.api.types.is_list_like
stock_dr = data.get_data_yahoo(sid+'.TW', start, end)
stock_dr.tail(10)

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2019-07-15,83.349998,82.250000,82.849998,83.199997,9336043.0,82.495766
2019-07-16,83.449997,83.150002,83.199997,83.349998,5503245.0,82.644493
2019-07-17,83.250000,82.800003,83.250000,82.800003,10598530.0,82.099159
2019-07-18,82.949997,82.599998,82.599998,82.699997,7541026.0,82.000000
2019-07-19,83.449997,83.000000,83.000000,83.000000,9780922.0,83.000000
2019-07-22,83.500000,83.099998,83.099998,83.400002,8368544.0,83.400002
2019-07-23,83.900002,83.500000,83.599998,83.599998,5899116.0,83.599998
2019-07-24,83.900002,83.550003,83.750000,83.550003,3364336.0,83.550003
2019-07-25,83.800003,83.400002,83.550003,83.800003,4931659.0,83.800003


In [4]:
stock_dr.columns=['high','low','open','close','volume','adj close']

In [105]:
sid = '2330'
data=twstock.Stock(sid)
df = pd.DataFrame(data.fetch_from(2019,7))
df.set_index('date', inplace = True)

In [125]:
df.tail()

,capacity,turnover,open,high,low,close,change,transaction
date,,,,,,,,
2019-07-22,31863639,8385222407,263.0,264.0,262.0,264.0,5.0,13424
2019-07-23,29273223,7734089131,265.5,266.5,262.5,264.0,0.0,12791
2019-07-24,24511039,6484708226,266.5,266.5,263.0,265.0,1.0,10617
2019-07-25,30698545,8091922380,264.0,265.0,261.5,265.0,0.0,10643
2019-07-26,20768071,5434082968,261.0,262.5,261.0,261.0,-4.0,8850


In [15]:
#print(abstract.MACD(df.close))
#len(df)
# print(abstract.MACD(stock_dr))
k=stock_dr.values[:25]
# stock_dr.tail()
np.size(k)

150

In [14]:
#收盤價
#df['close'].plot(figsize=(16, 8))
#KD
#abstract.STOCH(df).plot(figsize=(16, 8)) 
abstract.STOCH(k)
#MACD
#abstract.MACD(df).plot(figsize=(16, 8)) 
abstract.MACD()


TypeError: only size-1 arrays can be converted to Python scalars

In [7]:
#取前250個網址

url='https://www.wantgoo.com/stock/astock/techchart?StockNo=2324&m=dk'
res= requests.get(url)
soup=BeautifulSoup(res.text, "html.parser")
article=soup.find_all({"class":"highcharts-series"})
# url_list=[]
# for i in range(len(article)):
#     u1=article[i].a.get('href')
#     url_list+=['https://socialblade.com'+u1+'/monthly']
article

[]